In [1]:
import pandas as pd
from sqlalchemy import create_engine

## Extract

In [2]:
def extract_excels():
    try:
        path =  '../preprocessing/preprocessed files/'

        df_Airbnb = pd.read_excel(path + 'Final_Airbnb.xlsx')
        print("df_Airbnb succesfully loaded!")   

        df_Booking = pd.read_excel(path + 'Final_Booking.xlsx')
        print("df_Booking succesfully loaded!")  

        df_Oltp = pd.read_excel(path + 'Final_Oltp.xlsx')
        print("df_Oltp succesfully loaded!")

    except Exception as err:
        print('Error while connecting to the db')
        print(err) 

    return df_Airbnb, df_Booking, df_Oltp

In [3]:
df_Airbnb, df_Booking, df_Oltp = extract_excels()

df_Airbnb succesfully loaded!
df_Booking succesfully loaded!
df_Oltp succesfully loaded!


Merge all files into one

In [4]:
df_merged = pd.concat([df_Airbnb, df_Booking, df_Oltp]).sort_values("Start date")
df_merged["avg_rate_per_reservation"] = df_merged["Earnings after Tax"] / df_merged["# of nights"]
df_merged.head(3)

,Unnamed: 0,Status,Guest name,Origin,# of guests,Booked,Start date,End date,# of nights,# of days pre booked,Listing name,# of beds,Earnings,Tax,Earnings after Tax,Advertiser,Cancel date,# of days cancel,avg_rate_per_reservation
1,1,Confirmed,Youstina Daoud,Egypt,2,2019-03-30,2019-06-03,2019-06-05,2,65,Studio with Patio,2,67.90,4.0,63.90,Airbnb,0,0.0,31.95
6,6,Confirmed,Kristina Holm Jensen,Denmark,2,2019-05-05,2019-06-05,2019-06-07,2,31,Studio with Patio,2,67.90,4.0,63.90,Airbnb,0,0.0,31.95
2,2,Confirmed,Öznur Balaban,Turkey,2,2019-04-09,2019-06-08,2019-06-09,1,60,Studio with Patio,2,33.95,2.0,31.95,Airbnb,0,0.0,31.95


## Third DW theme - Transform

In [5]:
df = pd.DataFrame()

Dim_Apartment - dimension table

In [6]:
# init empty df with column names
dim_Apartment = pd.concat([df.copy(), pd.DataFrame(columns=['apartment_id', 'apartment_name'])], axis=1)

df_merged["apartment_id"] = df_merged["Listing name"].rank(method='dense').astype(int)

dim_Apartment = df_merged[['apartment_id', 'Listing name']].drop_duplicates().sort_values("apartment_id")
dim_Apartment.head()


,apartment_id,Listing name
7,1,One-Bedroom Apartment with Balcony and Sea View
22,2,One-Bedroom Apartment with Patio and Sea View
1,3,Studio with Patio
23,4,Studio with Patio and Sea View


Dim_Days_Prebooked - interval dimension table

In [7]:
Dim_Days_Prebooked_data = {
    'prebook_id': [1,2,3,4,5,6],
    'prebook_interval': ["0-2 days", "3-7 days", "8-14 days", "15-30 days", "31-60 days", "60+ days"],
    'date_from': [0, 3, 8, 15, 31, 61],
    'date_to': [2, 7, 14, 30, 60, 9999]
}

Dim_Days_Prebooked = pd.DataFrame(Dim_Days_Prebooked_data)
Dim_Days_Prebooked.head()

,prebook_id,prebook_interval,date_from,date_to
0,1,0-2 days,0,2
1,2,3-7 days,3,7
2,3,8-14 days,8,14
3,4,15-30 days,15,30
4,5,31-60 days,31,60


Dim_No_Days_Cancelled - interval dimension table

In [8]:
Dim_No_Days_Cancelled_data = {
    'cancel_id': [1,2,3,4,5,6],
    'cancel_interval': ["0-2 days", "3-7 days", "8-14 days", "15-30 days", "31-60 days", "60+ days"],
    'date_from': [0, 3, 8, 15, 31, 61],
    'date_to': [2, 7, 14, 30, 60, 9999]
}

Dim_No_Days_Cancelled = pd.DataFrame(Dim_No_Days_Cancelled_data)
Dim_No_Days_Cancelled.head()

,cancel_id,cancel_interval,date_from,date_to
0,1,0-2 days,0,2
1,2,3-7 days,3,7
2,3,8-14 days,8,14
3,4,15-30 days,15,30
4,5,31-60 days,31,60


Dim_Time - dimension table

In [9]:
# init date range
dates = pd.date_range(start='2018-01-01', end='2023-12-31')

dim_Dates = pd.DataFrame({'date': dates, 'day': dates.day, 'month': dates.month, 'quarter': dates.quarter, 'year': dates.year})

dim_Dates['week_of_year'] = dim_Dates['date'].dt.isocalendar().week
dim_Dates['month_name'] = dim_Dates['date'].dt.strftime('%B')

dim_Dates['day_of_week'] = dim_Dates['date'].dt.weekday
dim_Dates['day_name'] = dim_Dates['date'].dt.strftime('%A')

# set indexes
dim_Dates = dim_Dates.reset_index(drop = True)
dim_Dates['date_id'] = dim_Dates.index + 1

dim_Dates.head(3)

,date,day,month,quarter,year,week_of_year,month_name,day_of_week,day_name,date_id
0,2018-01-01,1,1,1,2018,1,January,0,Monday,1
1,2018-01-02,2,1,1,2018,1,January,1,Tuesday,2
2,2018-01-03,3,1,1,2018,1,January,2,Wednesday,3


Fct_Cancelation_Policy - fact table

In [10]:
fct_Cancelation_Policy = df_merged.copy()

fct_Cancelation_Policy.rename(columns={"Start date": "date"}, inplace=True)
fct_Cancelation_Policy = fct_Cancelation_Policy.merge(dim_Dates[["date_id", "date"]], on='date', how='left')

fct_Cancelation_Policy = fct_Cancelation_Policy.reset_index(drop = True)
fct_Cancelation_Policy['cancelation_id'] = fct_Cancelation_Policy.index + 1

fct_Cancelation_Policy = fct_Cancelation_Policy[["cancelation_id", "apartment_id", "date_id", "# of days pre booked", 'avg_rate_per_reservation', "Earnings after Tax", "# of days cancel", "Status"]]
fct_Cancelation_Policy.head(3)

,cancelation_id,apartment_id,date_id,# of days pre booked,avg_rate_per_reservation,Earnings after Tax,# of days cancel,Status
0,1,3,519,65,31.95,63.90,0.0,Confirmed
1,2,3,521,31,31.95,63.90,0.0,Confirmed
2,3,3,524,60,31.95,31.95,0.0,Confirmed


In [11]:
def replace_values(num):
    if 0 <= num <= 2:
        return 1
    elif 3 <= num <= 7:
        return 2
    elif 8 <= num <= 14:
        return 3
    elif 15 <= num <= 30:
        return 4
    elif 31 <= num <= 60:
        return 5
    else:
        return 6

In [12]:
fct_Cancelation_Policy["prebook_id"] = fct_Cancelation_Policy["# of days pre booked"].apply(replace_values)
fct_Cancelation_Policy["cancel_id"] = fct_Cancelation_Policy["# of days cancel"].apply(replace_values)

fct_Cancelation_Policy.drop(columns=["# of days pre booked", "# of days cancel"], inplace=True)
fct_Cancelation_Policy.rename(columns={"cancelation_id": "cancelation_pol_id","Earnings after Tax": "total_price"}, inplace=True)

fct_Cancelation_Policy.head(3)

,cancelation_pol_id,apartment_id,date_id,avg_rate_per_reservation,total_price,Status,prebook_id,cancel_id
0,1,3,519,31.95,63.90,Confirmed,6,1
1,2,3,521,31.95,63.90,Confirmed,5,1
2,3,3,524,31.95,31.95,Confirmed,5,1


In [13]:
fct_Cancelation_Policy = fct_Cancelation_Policy[["cancelation_pol_id", "apartment_id", "date_id", "prebook_id", "cancel_id", "Status", "avg_rate_per_reservation", "total_price"]] 

# we only need cancelled and confirmed reservations
fct_Cancelation_Policy = fct_Cancelation_Policy[fct_Cancelation_Policy["Status"].isin(["Confirmed", "Cancelled"])]
fct_Cancelation_Policy.head(3)

,cancelation_pol_id,apartment_id,date_id,prebook_id,cancel_id,Status,avg_rate_per_reservation,total_price
0,1,3,519,6,1,Confirmed,31.95,63.90
1,2,3,521,5,1,Confirmed,31.95,63.90
2,3,3,524,5,1,Confirmed,31.95,31.95


Checking how many Cancelled and Confirmed reservations there are

In [14]:
value_counts = fct_Cancelation_Policy["Status"].value_counts()
value_counts

Confirmed    247
Cancelled      5
Name: Status, dtype: int64

## Load tables to DW

Defining function to make connection to PostgreSQL server

In [15]:
def load(df, tbl):
    try:
        rows_imported = 0
        engine = create_engine(f'postgresql://{uid}:{pwd}@{server}:5432/DW_Reservation_System')
        print(f'importing rows {rows_imported} to {rows_imported + len(df)}... for table {tbl}')
        # save df to postgres
        df.to_sql(f'{tbl}', engine, if_exists = "replace", index = False)
        rows_imported += len(df)
        print('Data imported sucessfully!')
    except Exception as e:
        print('Data load error: ' + str(e))

Forwarding data frames, table names and connection parameters

In [16]:
uid, pwd, server = "etl", "pass", "localhost"

# we dont need time and apartment tables, as thei were already loaded in first theme
dataframes_to_load = [ Dim_Days_Prebooked, Dim_No_Days_Cancelled, fct_Cancelation_Policy]
table_names = ["Dim_Days_Prebooked", "Dim_No_Days_Cancelled", "fct_Cancelation_Policy"]

In [17]:
for df, tbl in zip(dataframes_to_load, table_names):
    load(df, tbl)

importing rows 0 to 6... for table Dim_Days_Prebooked
Data imported sucessfully!
importing rows 0 to 6... for table Dim_No_Days_Cancelled
Data imported sucessfully!
importing rows 0 to 252... for table fct_Cancelation_Policy
Data imported sucessfully!
